In [ ]:
import os
import sys
import json
import math
from astropy.time import Time as astrotime
from tqdm import tqdm_notebook as tqdm

import numpy as np
from numpy.lib.recfunctions import drop_fields, stack_arrays

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.integrate as scint
import scipy.stats as scs

import anapymods3.plots.astro as amp_plt
from anapymods3.plots import (split_axis, get_binmids, hist_marginalize,
                              hist_from_counts, dg)
from anapymods3.stats import json2kde, sigma2prob, percentile_nzeros
from anapymods3.general import fill_dict_defaults, total_size
from anapymods3.healpy import wrap_theta_phi_range

import tdepps.bg_injector as BGInj
import tdepps.bg_rate_injector as BGRateInj
import tdepps.rate_function as RateFunc
import tdepps.llh as LLH
import tdepps.analysis as Analysis
import tdepps.signal_injector as SigInj
from tdepps.utils import (func_min_in_interval, rotator,
                          make_ns_poisson_weights, weighted_cdf, delta_chi2)

secinday = 24. * 60. * 60.
print("Executed on ", astrotime.now())

Test multi year analysis build on top of the single year one.

# Setup GRB samples

In [ ]:
rndgen_86II = np.random.RandomState(735005)  # Google
rndgen_86III = np.random.RandomState(7353)   # Esel

### IC86III (2013)

In [ ]:
data_86III = np.load("data/GRB_IC86III/data.npy")
sim_86III = np.load("data/GRB_IC86III/sim_IC86III_nugen_10634.npy")
grblist_86III = np.load("data/GRB_IC86III/grblist.npy")
with open("data/GRB_IC86III/runlist.json", "r") as infile:
    runlist_86III = json.load(infile)

# Number of files is different in both years, see weight check later
nfiles_86III = 956
nevents = 5000
ngen_86III = float(nfiles_86III * nevents)
    
# Cut on score
score_cut_86III = 0.5
data_86III = data_86III[data_86III["score"] > score_cut_86III]
sim_86III = sim_86III[sim_86III["score"] > score_cut_86III]
    
run_ids = runlist_86III.keys()
livetimes = []
for key in run_ids:
    livetimes.append(runlist_86III[key]["livetime"])
livetimes_86III = np.array(livetimes)
livetime_86III = np.sum(livetimes)
livetime_days_86III = livetime_86III / 24. / 3600.

In [ ]:
def is_on_time(run_id):
    key = str(int(run_id))
    return True if runlist_86III[key]["ontime"] else False

on_mask = np.array(list(map(is_on_time, data_86III["run_id"])))
assert np.sum(on_mask) + np.sum(~on_mask) == len(data_86III)

on_data_86III = data_86III[on_mask]
off_data_86III = data_86III[~on_mask]

# Get livetime split
on_livetime_86III = np.sum([run["livetime"] if run["ontime"] else 0.
                            for run in runlist_86III.values()])
off_livetime_86III = np.sum([run["livetime"] if not run["ontime"] else 0.
                            for run in runlist_86III.values()])
assert livetime_86III == (on_livetime_86III + off_livetime_86III)

print("Off livetime   : {:6.2f} days".format(off_livetime_86III / 24. / 3600.))
print("On livetime    : {:6.2f} days".format(on_livetime_86III / 24. / 3600.))
print("Total livetime : {:6.2f} days".format(livetime_days_86III))

In [ ]:
nsrcs = len(grblist_86III)
names = ["t", "dt0", "dt1", "ra", "dec", "w_theo", "sigma"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]

grb_srcs_IC86III = np.empty((nsrcs, ), dtype=dtype)

grb_srcs_IC86III["t"] = grblist_86III["timeMJD"]
grb_srcs_IC86III["dt0"] = np.zeros(nsrcs, dtype=np.float)
grb_srcs_IC86III["dt1"] = grblist_86III["t100"]
grb_srcs_IC86III["ra"] = grblist_86III["ra"]
grb_srcs_IC86III["dec"] = grblist_86III["dec"]
grb_srcs_IC86III["w_theo"] = np.ones(nsrcs, dtype=np.float)
grb_srcs_IC86III["sigma"] = grblist_86III["sigma"]

In [ ]:
runs = []
for key, item in runlist_86III.items():
    d = item.copy()
    tstart = astrotime(item["tStart"], format="mjd").iso
    tstop = astrotime(item["tStop"], format="mjd").iso
    d["run"] = int(key)
    d["good_tstart"] = tstart
    d["good_tstop"] = tstop
    runs.append(d)

goodruns_86III = {"runs": runs}
print("Number of all runs : ", len(runs))
off_len =  len(filter(lambda runi: not runi["ontime"], runs))
on_len =  len(filter(lambda runi: runi["ontime"], runs))
print("Number of off runs : ",off_len)
print("Number of on runs  : ", on_len)

assert off_len + on_len == len(runs)

In [ ]:
# Create the GRBLLH object with all the PDF settings
# Note: Range derived from sinDec hist in NRT wiki
sin_dec_bins = np.linspace(-0.1, 1, 17 + 1)  

# Range is derived from hist, dropping some wrongly reconstructed events
min_logE = -1.
max_logE = 10.
logE_bins = np.linspace(min_logE, max_logE, 40)

# Remove wrongly reconstructed events outside range from off data & sim
logE_mask = (sim_86III["logE"] > min_logE) & (sim_86III["logE"] < max_logE)
sim_pdf_full_86III = sim_86III[logE_mask]
print("Removed {} events in logE for simulation.".format(np.sum(~logE_mask)))
logE_mask = ((off_data_86III["logE"] > min_logE) &
             (off_data_86III["logE"] < max_logE))
data_pdf_full_86III = off_data_86III[logE_mask]
print("Removed {} events in logE for data.".format(np.sum(~logE_mask)))

# Later 3 events show up, that have times outside the run times. These are
# removed too - still strange.
time_idx = [39023, 52880, 63524]
data_pdf_full = np.delete(data_pdf_full_86III, time_idx)

# Check weights for E2 signal against built-in weights
gamma = 2.
type_w = 0.5         # Nugen defaults: 50:50 nu, anti-nu
norm_per_type = 0.5  # Astro just splits 50:50
# Norm to OW per type and "sim-livetime"
sim_pdf_full_86III["ow"] /= ngen_86III * type_w
w = (norm_per_type * sim_pdf_full_86III["trueE"]**(-gamma) *
     sim_pdf_full_86III["ow"])

print("Included and self made weights ar the same: ",
      np.allclose(sim_pdf_full_86III["weight_E2"], w))

# Now strip off uneeded fields so data and sim are compatible for tdepps
data_pdf_86III = drop_fields(data_pdf_full_86III,
                             ["run_id", "event_id", "score", "azi", "zen"])
sim_pdf_86III = drop_fields(sim_pdf_full_86III,
                            ["azi", "zen", "score", "weight_E2",
                             "weight_honda", "trueAzi", "trueZen"])

### IC86II (2012)

In [ ]:
data_86II = np.load("data/GRB_IC86II/data.npy")
sim_86II = np.load("data/GRB_IC86II/sim_IC86II_nugen_10634.npy")
grblist_86II = np.load("data/GRB_IC86II/grblist.npy")
with open("data/GRB_IC86II/runlist.json", "r") as infile:
    runlist_86II = json.load(infile)

# Number of files is different in both years, see weight check later
nfiles_86II = nfiles_86III + 5
nevents = 5000
ngen_86II = float(nfiles_86II * nevents)
    
# Cut on score
score_cut_86II = 0.4
data_86II = data_86II[data_86II["score"] > score_cut_86II]
sim_86II = sim_86II[sim_86II["score"] > score_cut_86II]
    
run_ids = runlist_86II.keys()
livetimes = []
for key in run_ids:
    livetimes.append(runlist_86II[key]["livetime"])
livetimes_86II = np.array(livetimes)
livetime_86II = np.sum(livetimes)
livetime_days_86II = livetime_86II / 24. / 3600.

In [ ]:
def is_on_time(run_id):
    key = str(int(run_id))
    return True if runlist_86II[key]["ontime"] else False

on_mask = np.array(list(map(is_on_time, data_86II["run_id"])))
assert np.sum(on_mask) + np.sum(~on_mask) == len(data_86II)

on_data_86II = data_86II[on_mask]
off_data_86II = data_86II[~on_mask]

# Get livetime split
on_livetime_86II = np.sum([run["livetime"] if run["ontime"] else 0.
                            for run in runlist_86II.values()])
off_livetime_86II = np.sum([run["livetime"] if not run["ontime"] else 0.
                            for run in runlist_86II.values()])
assert livetime_86II == (on_livetime_86II + off_livetime_86II)

print("Off livetime   : {:6.2f} days".format(off_livetime_86II / 24. / 3600.))
print("On livetime    : {:6.2f} days".format(on_livetime_86II / 24. / 3600.))
print("Total livetime : {:6.2f} days".format(livetime_days_86II))

In [ ]:
nsrcs = len(grblist_86II)
names = ["t", "dt0", "dt1", "ra", "dec", "w_theo", "sigma"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]

grb_srcs_IC86II = np.empty((nsrcs, ), dtype=dtype)

grb_srcs_IC86II["t"] = grblist_86II["timeMJD"]
grb_srcs_IC86II["dt0"] = np.zeros(nsrcs, dtype=np.float)
grb_srcs_IC86II["dt1"] = grblist_86II["t100"]
grb_srcs_IC86II["ra"] = grblist_86II["ra"]
grb_srcs_IC86II["dec"] = grblist_86II["dec"]
grb_srcs_IC86II["w_theo"] = np.ones(nsrcs, dtype=np.float)
grb_srcs_IC86II["sigma"] = grblist_86II["sigma"]

In [ ]:
runs = []
for key, item in runlist_86II.items():
    d = item.copy()
    tstart = astrotime(item["tStart"], format="mjd").iso
    tstop = astrotime(item["tStop"], format="mjd").iso
    d["run"] = int(key)
    d["good_tstart"] = tstart
    d["good_tstop"] = tstop
    runs.append(d)

goodruns_86II = {"runs": runs}
print("Number of all runs : ", len(runs))
off_len =  len(filter(lambda runi: not runi["ontime"], runs))
on_len =  len(filter(lambda runi: runi["ontime"], runs))
print("Number of off runs : ",off_len)
print("Number of on runs  : ", on_len)

assert off_len + on_len == len(runs)

In [ ]:
# Create the GRBLLH object with all the PDF settings
# Note: Range derived from sinDec hist in NRT wiki
sin_dec_bins = np.linspace(-0.1, 1, 17 + 1)  

# Range is derived from hist, dropping some wrongly reconstructed events
min_logE = -1.
max_logE = 10.
logE_bins = np.linspace(min_logE, max_logE, 40)

# Remove wrongly reconstructed events outside range from off data & sim
logE_mask = (sim_86II["logE"] > min_logE) & (sim_86II["logE"] < max_logE)
sim_pdf_full_86II = sim_86II[logE_mask]
print("Removed {} events in logE for simulation.".format(np.sum(~logE_mask)))
logE_mask = ((off_data_86II["logE"] > min_logE) &
             (off_data_86II["logE"] < max_logE))
data_pdf_full_86II = off_data_86II[logE_mask]
print("Removed {} events in logE for data.".format(np.sum(~logE_mask)))

# Later 3 events show up, that have times outside the run times. These are
# removed too - still strange.
# time_idx = [39023, 52880, 63524]
# data_pdf_full = np.delete(data_pdf_full_86II, time_idx)

# Check weights for E2 signal against built-in weights
gamma = 2.
type_w = 0.5         # Nugen defaults: 50:50 nu, anti-nu
norm_per_type = 0.5  # Astro just splits 50:50
sim_pdf_full_86II["ow"] /= ngen_86II * type_w  # Norm to OW per type and "sim-livetime"
w = (norm_per_type * sim_pdf_full_86II["trueE"]**(-gamma) *
     sim_pdf_full_86II["ow"])

print("Included and self made weights ar the same: ",
      np.allclose(sim_pdf_full_86II["weight_E2"], w))

# Now strip off uneeded fields so data and sim are compatible for tdepps
data_pdf_86II = drop_fields(data_pdf_full_86II,
                             ["run_id", "event_id", "score", "azi", "zen"])
sim_pdf_86II = drop_fields(sim_pdf_full_86II,
                            ["azi", "zen", "score", "weight_E2",
                             "weight_honda", "trueAzi", "trueZen"])

### Done:

In [ ]:
print("Setup done")

# tdepps setup

In [ ]:
spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}

energy_pdf_args_86III = {"bins": [sin_dec_bins, logE_bins], "gamma": 2.,
                         "fillval": "col", "interpol_log": False,
                         "mc_bg_weights": sim_pdf_full_86III["weight_honda"],
                         "smooth_sigma": [[0., 0.], [0., 0.]],
                         "logE_asc": True}
energy_pdf_args_86II = {"bins": [sin_dec_bins, logE_bins], "gamma": 2.,
                        "fillval": "col", "interpol_log": False,
                        "mc_bg_weights": sim_pdf_full_86II["weight_honda"],
                        "smooth_sigma": [[0., 0.], [0., 0.]],
                        "logE_asc": True}

time_pdf_args = {"nsig": 4., "sigma_t_min": 2., "sigma_t_max": 30.}

llh_args = {"sob_rel_eps": 0., "sob_abs_eps": 1e-4}

grbllh_ic86III = LLH.GRBLLH(X=data_pdf_86III, MC=sim_pdf_86III,
                            spatial_pdf_args=spatial_pdf_args,
                            energy_pdf_args=energy_pdf_args_86III,
                            time_pdf_args=time_pdf_args,
                            llh_args=llh_args)

grbllh_ic86II = LLH.GRBLLH(X=data_pdf_86II, MC=sim_pdf_86II,
                            spatial_pdf_args=spatial_pdf_args,
                            energy_pdf_args=energy_pdf_args_86II,
                            time_pdf_args=time_pdf_args,
                            llh_args=llh_args)

In [ ]:
multillh2 = LLH.MultiSampleGRBLLH()
multillh2.add_sample("86III", grbllh_ic86III)
multillh2.add_sample("86II", grbllh_ic86II)

BG Injectors

In [ ]:
# Choose your rate function
rate_func_obj_86III = RateFunc.Sinus1yrConstRateFunction(rndgen_86III)
rate_func_obj_86II = RateFunc.Sinus1yrConstRateFunction(rndgen_86II)

# Make the injector based on the rate function
bg_rate_inj_86III = BGRateInj.RunlistBGRateInjector(
    rate_func_obj_86III, goodruns_86III, random_state=rndgen_86III)
bg_rate_inj_86II = BGRateInj.RunlistBGRateInjector(
    rate_func_obj_86II, goodruns_86II, random_state=rndgen_86II)

# Fit the injector to make it usable
times_86III = np.delete(data_pdf_86III["timeMJD"], [39023, 52880, 63524])
rate_func_86III = bg_rate_inj_86III.fit(T=times_86III, x0=None,
                                        remove_zero_runs=False)
out_times = [1455, 7030, 8863, 13724, 17655, 20423, 22191, 26127, 28365,
             45350, 48338, 53590, 104330, 120422, 123279]
times_86II = np.delete(data_pdf_86II["timeMJD"], out_times)
rate_func_86II = bg_rate_inj_86II.fit(T=times_86II, x0=None,
                                      remove_zero_runs=False)

# Now the evt injector for the other attributes
# bg_inj = BGInj.MRichmanBGInjector(random_state=rndgen)
bg_inj_86III = BGInj.DataBGInjector(random_state=rndgen_86III)
bg_inj_86III.fit(X=data_pdf_86III)
bg_inj_86II = BGInj.DataBGInjector(random_state=rndgen_86II)
bg_inj_86II.fit(X=data_pdf_86II)

# Multi injectors
multibginj = BGInj.MultiBGInjector()
multibgrateinj = BGRateInj.MultiBGRateInjector()

multibginj.add_injector("86II", bg_inj_86II)
multibginj.add_injector("86III", bg_inj_86III)

multibgrateinj.add_injector("86II", bg_rate_inj_86II)
multibgrateinj.add_injector("86III", bg_rate_inj_86III)

Signal Injector

In [ ]:
siginj = SigInj.SignalInjector(gamma=2., sin_dec_range=sin_dec_bins[[0, -1]])
mcs = {"86III": sim_pdf_86III, "86II": sim_pdf_86II}
srcs = {"86III": grb_srcs_IC86III, "86II": grb_srcs_IC86II}
siginj.fit(srcs=srcs, MC=mcs, exp_names=data_pdf_86III.dtype.names)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj.mu2flux(1)))

sig_gen = siginj.sample(mean_mu=10)

## Use it

In [ ]:
srcs = {"86III": grb_srcs_IC86III, "86II": grb_srcs_IC86II}
ana = Analysis.TransientsAnalysis(srcs=srcs, llh=multillh2)

Scan for 90% over TS=0

In [ ]:
ts_val = 0.
beta = 0.9
bg_inj = multibginj
verb = True
tol_perc_err = 5e-3
tol_mu_rel = 1e-3
maxloops = 100

perf = ana.performance(ts_val, beta, multibginj, multibgrateinj, siginj,
                       mu0=-0.1, ntrials=200, tol_perc_err=tol_perc_err,
                       tol_mu_rel=tol_mu_rel, maxloops=maxloops,
                       minimizer_opts=None, verb=verb)

In [ ]:
mu_bf = perf["mu_bf"]
mus = perf["mus"]
ns = perf["ns"]
ts = perf["ts"]
poisson_limit = 2.3

fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(12, 8))

# Plot the mu evolution
axtl.plot(perf["mus"])
axtl.axhline(poisson_limit, 0, 1, color="C1", ls="--", label="Poisson Lim.")
axtl.axvline(perf["ninitloops"] - 1, 0, 1, color="C7", ls="--",
             label="Init loops")
axtl.set_xlabel("Iteration i")
axtl.set_ylabel("mu")
axtl.set_title("Best fit after iteration. Best fit: {:.3f}".format(mu_bf))
axtl.legend(loc="best")

# Plot the rel/abs error evolution
axtr.plot(perf["err"], color="C0", label="perc err")
axtr.plot(np.abs(np.diff(mus)) / mus[1:], color="C1", label="mu diff abs")
axtr.axhline(tol_perc_err, 0, 1, color="C0", ls="--", label="tol perc")
axtr.axhline(tol_mu_rel, 0, 1, color="C1", ls="--", label="mu diff tol")
axtr.axvline(perf["ninitloops"] - 1, 0, 1, color="C7", ls="--",
             label="Init loops")
axtr.set_xlabel("Iteration i")
axtr.set_yscale("log", nonposy="clip")
axtr.set_title("Performance after ith Iteration")
axtr.legend(loc="best")

# Plot all drawn, unweighted ns values
_ = axbl.hist(ns, bins=200)
axbl.axvline(poisson_limit, 0, 1, color="C1", ls="--", label="Poisson Lim.")
axbl.set_xlabel("ns")
axbl.set_ylabel("Count")
axbl.set_title("Injected ns unweighted")
axbl.legend(loc="best")

# Plot all drawn, weighted test statistic values
w, _ = make_ns_poisson_weights(mu_bf, ns)
perc = 1. - weighted_cdf(ts, ts_val, w)[0]  # want the perc above TS val
bins = np.arange(0., 20., 0.25)
_ = axbr.hist(ts, bins=bins, weights=w)
axbr.axvline(ts_val, 0, 1, color="C1", ls="--", label="TS val")
axbr.set_xlabel("TS")
axbr.set_ylabel("PDF")
axbr.set_title("Poisson Weighted TS. Percentile = {:.6f}".format(perc))
axbr.legend(loc="best")

plt.tight_layout()
# plt.savefig("./fig_new_bg/performance_poisson_reweight.png", dpi=150)
plt.show()

Make BG trials

In [ ]:
ntrials = 100000
ns0 = 0.5
res_bg2, nz_bg2, _ = ana.do_trials(ntrials, ns0,
                       bg_inj=multibginj, bg_rate_inj=multibgrateinj,
                       signal_inj=None, full_out=True, verb=True)

In [ ]:
with open("./data/GRB_IC86multi/bg_trials.json", "w") as f:
    out = {"nzeros": nz_bg, "TS": list(res_bg["TS"]),
           "ns": list(res_bg["ns"])}
    json.dump(obj=out, fp=f, indent=2)

In [ ]:
bins = np.arange(0, 30, 0.25)
h_bg, _ = np.histogram(res_bg["TS"], bins=bins, density=False)
h_bg[0] += nz_bg
err_bg = np.sqrt(h_bg)
norm = (np.diff(bins) * np.sum(h_bg))
h_bg = h_bg / norm
err_bg = err_bg / norm

df, eta, loc, scale = delta_chi2.fit_nzeros(data=res_bg["TS"], nzeros=nz_bg)

TS = np.linspace(bins[0], bins[-1], 200)
PDF = delta_chi2.pdf(TS, df, eta, loc, scale)
plt.plot(TS, PDF)
plt.plot(bins, np.r_[h_bg[0], h_bg], drawstyle="steps-pre", color="C1")
plt.errorbar(0.5 * (bins[:-1] + bins[1:]), h_bg, yerr=err_bg,
             fmt="none", color="C1")
plt.xlabel("TS")
plt.yscale("log", nonposy="clip")
plt.ylim(1e-6, 1)
plt.tight_layout()
# plt.savefig("./data/GRB_IC86multi/GRB86_multi_bg_trials.png", dpi=150)
plt.show()

Neyman plane

In [ ]:
# %%time
ntrials_sig = int(1e3)
ns0 = 0.5
minopts = {"bounds": [[0., None]]}

res_sig = []
nzeros_sig = []
mu_sig = np.arange(0.1, 10.1, 0.5)
for mui in mu_sig:
    print("Start injecting with {:.1f}".format(mui))
    gen = siginj.sample(mean_mu=mui, poisson=True)
    r, nz = ana.do_trials(ntrials_sig, ns0=ns0, bg_inj=multibginj,
                          bg_rate_inj=multibgrateinj, signal_inj=gen,
                          minimizer_opts=minopts, verb=True)
    res_sig.append(r)
    nzeros_sig.append(nz)

print("Done")

In [ ]:
out = {}
for i, mui in enumerate(mu_sig):
    out[mui] = {"TS": list(res_sig[i]["TS"]),
                "ns": list(res_sig[i]["ns"]),
                "nzeros": nzeros_sig[i]}
    
with open("./data/GRB_IC86multi/neyman_sweep.json", "w") as f:
    json.dump(obj=out, fp=f, indent=2)

In [ ]:
TS_bg = res_bg["TS"]
nzeros_bg = nz_bg

# Choose bins so that mids cover the injected mu
b_TS = np.arange(0, 50 + 0.25, 0.25)
b_mu = np.r_[0., get_binmids(np.r_[0., mu_sig]),
             mu_sig[-1] + 0.5 * np.diff(mu_sig[-2:])]
b_flux = siginj.mu2flux(b_mu)

# Bin mids are the mus we injected at, including BG only at mu=0
m_TS = get_binmids(b_TS)
m_mu = np.r_[0., mu_sig]
m_flux = siginj.mu2flux(m_mu)

# Create pseudo coordinates to histogram with weights
XX, YY_mu = np.meshgrid(m_TS, m_mu)
XX, YY_flux = np.meshgrid(m_TS, m_flux)

# Build up a 2D hist from each 1D trial hist per mu
shape = (len(m_mu), len(m_TS))
h_all = np.zeros(shape, dtype=np.float)

# BG only, add in nzeros and normalize after that
h_bg, _ = np.histogram(TS_bg, bins=b_TS, normed=False)
h_bg[0] += nzeros_bg
h_bg = h_bg / (np.diff(b_TS) * np.sum(h_bg))
h_all[0] = h_bg

# Signal sweep
for i, (res_i, nz_i) in enumerate(zip(res_sig, nzeros_sig)):
    h_sig, _ = np.histogram(res_i["TS"], bins=b_TS, normed=False)
    h_sig[0] += nz_i
    h_sig = h_sig / (np.diff(b_TS) * np.sum(h_sig))
    h_all[i+1] = h_sig

In [ ]:
# Get PDF stats for each slice
fluxes = siginj.mu2flux(mu_sig)
upper_lim_perc = 0.9
sigma = [3, 4, 5]
perc_bg = [percentile_nzeros(TS_bg, nzeros_bg, sigma2prob(sig) * 100) for
           sig in sigma]

UL = []
for i, (res_i, nz_i) in enumerate(zip(res_sig, nzeros_sig)):
    UL.append(percentile_nzeros(res_i["TS"], nz_i,
                                100 * (1. - upper_lim_perc)))

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(14, 5))
ylabel = [r"Injected mean events $\mu$",
          r"$(E/\mathrm{GeV})^2\times F$ in (GeV cm$^2$)$^{-1}$"]
for x, YY, ylab, ax in zip([mu_sig, fluxes], [YY_mu, YY_flux], ylabel,
                           (axl, axr)):
    img = ax.pcolormesh(XX, YY, h_all, norm=LogNorm(), cmap="inferno")

    # Ignore BG TS, because it's not done with energy PDF from MC
    alpha = 0.5
    for sig, pbg in zip(sigma, perc_bg):
        ax.axvline(pbg, 0, 1, color="k", lw=1.5, ls="--", alpha=alpha,
                   label=r"${}\sigma$ wrt BG".format(sig))
        alpha += 0.2
        
    ax.plot(UL, x, color="w", lw=2, label=r"90% UL per $\mu$")

    ax.set_xlabel("TS")
    ax.set_ylabel(ylab)
    ax.set_title("Neyman construction")
    cbar = fig.colorbar(img, ax=ax)
    cbar.set_label("PDF")
    ax.set_xlim(b_TS[0], b_TS[-1])
    ax.legend(loc="upper right")

plt.tight_layout()
# plt.savefig("./data/GRB_IC86multi/GRB86_multi_neyman_plane.png", dpi=150)
plt.show()

# Single modules tests

## LLH evaluation

In [ ]:
# Create data to evaluate from both on-datasets
names = ["ra", "dec", "sinDec", "logE", "sigma", "timeMJD"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]
X_86III = np.empty((len(on_data_86III)), dtype=dtype)
X_86II = np.empty((len(on_data_86II)), dtype=dtype)

for name in names:
    X_86III[name] = on_data_86III[name]
    X_86II[name] = on_data_86II[name]

In [ ]:
# Approximate BG expectation per GRB: 5mHz * dt, equal for both
rate_86III = len(off_data_86III) / off_livetime_86III
rate_86II = len(off_data_86II) / off_livetime_86II

nb_86III = rate_86III * (grb_srcs_IC86III["dt1"] - grb_srcs_IC86III["dt0"])
nb_86II = rate_86II * (grb_srcs_IC86II["dt1"] - grb_srcs_IC86II["dt0"])

### 2 samples

In [ ]:
multillh2 = LLH.MultiSampleGRBLLH()
multillh2.add_sample("86III", grbllh_ic86III)
multillh2.add_sample("86II", grbllh_ic86II)
X_dict = {"86III": X_86III, "86II": X_86II}
args_dict = {"86III": {"nb": nb_86III, "srcs": grb_srcs_IC86III},
             "86II": {"nb": nb_86II, "srcs": grb_srcs_IC86II}}

Add in some signal samples

In [ ]:
siginj = SigInj.SignalInjector(gamma=2.)
mcs = {"86III": sim_pdf_86III, "86II": sim_pdf_86II}
srcs = {"86III": grb_srcs_IC86III, "86II": grb_srcs_IC86II}
siginj.fit(srcs=srcs, MC=mcs, exp_names=data_pdf_86III.dtype.names)
gen = siginj.sample(mean_mu=10)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj.mu2flux(1)))

In [ ]:
X_new = {}
sam = next(gen)

for key, arr in X_dict.items():
    X_new[key] = np.concatenate((arr, sam[1][key]))

Fit single and combined LLHs.

In [ ]:
grbllh_ic86II.fit_lnllh_ratio(X=X_new["86II"], ns0=1.,
                              args=args_dict["86II"],
                              bounds=[[0, None]], minimizer_opts=None)

In [ ]:
grbllh_ic86III.fit_lnllh_ratio(X=X_new["86III"], ns0=1.,
                               args=args_dict["86III"],
                               bounds=[[0, None]], minimizer_opts=None)

Combined result should almost be the same if 

In [ ]:
ns_bf, TS_bf = multillh2.fit_lnllh_ratio(X=X_new, ns0=1., args=args_dict,
                                         bounds=[[0, None]],
                                         minimizer_opts=None)
print("ns best fit: {:.5f}".format(ns_bf))
print("TS best fit: {:.5f}".format(TS_bf))

Scan and plot LLH to verifiy best fit and gradient

In [ ]:
ns = np.linspace(0, 2 * ns_bf, 100)
llh2, grad2 = [], []
for nsi in tqdm(ns):
    llhi, gradi = multillh2.lnllh_ratio(X_new, nsi, args_dict)
    llh2.append(llhi)
    grad2.append(gradi)

llh2, grad2 = np.vstack((llh2, np.ravel(grad2)))

In [ ]:
plt.plot(ns, llh2)
plt.axvline(ns_bf, 0, 1, ls="--", c="k")
plt.xlabel("ns")
plt.xlabel("llh value")
plt.show()
plt.plot(ns, grad2)
plt.axhline(0, 0, 1, ls="--", c="k")
plt.axvline(ns_bf, 0, 1, ls="--", c="k")
plt.ylim(-2, 2)
plt.xlabel("llh gradient")
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh2) / np.diff(ns), ls="--")
plt.show()

### 1 sample

In [ ]:
multillh1 = LLH.MultiSampleGRBLLH()
multillh1.add_sample("86III", grbllh_ic86III)
X_dict = {"86III": X_86III}
args_dict = {"86III": {"nb": nb_86III, "srcs": grb_srcs_IC86III}}

In [ ]:
ns = np.linspace(0, 10, 100)
llh1, grad1 = [], []
for nsi in tqdm(ns):
    llhi, gradi = multillh1.lnllh_ratio(X_dict, nsi, args_dict)
    llh1.append(llhi)
    grad1.append(gradi)

llh1, grad1 = np.vstack((llh1, np.ravel(grad1)))

In [ ]:
plt.plot(ns, llh1)
plt.show()
plt.plot(ns, grad1)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh1) / np.diff(ns), ls="--")
plt.show()

### Single sample LLH must be the same as multi LLH for a single added sample

In [ ]:
args = {"nb": nb_86III, "srcs": grb_srcs_IC86III}

In [ ]:
ns = np.linspace(0, 10, 100)
llh0, grad0 = [], []
for nsi in tqdm(ns):
    llhi, gradi = grbllh_ic86III.lnllh_ratio(X_86III, nsi, args)
    llh0.append(llhi)
    grad0.append(gradi)

llh0, grad0 = np.vstack((llh0, np.ravel(grad0)))

In [ ]:
plt.plot(ns, llh0)
try:
    plt.plot(ns, llh1, ls="--")
except:
    print("llh1 cell not executed yet")
plt.show()
plt.plot(ns, grad0)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh0) / np.diff(ns), ls="--")
try:
    plt.plot(ns, grad1, ls=":", lw=1)
except:
    print("grad1 cell not executed yet")
plt.show()

### LLH test plots

In [ ]:
sindec = np.linspace(-0.1, 1., 100)

yIII = grbllh_ic86III._pdf_spatial_background(ev_sin_dec=sindec)
yII = grbllh_ic86II._pdf_spatial_background(ev_sin_dec=sindec)

plt.plot(sindec, yIII, label="IC86III")
plt.plot(sindec, yII, label="IC86II")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

bins = [sin_dec_bins, logE_bins]
x = np.linspace(*bins[0][[0, -1]], num=100 + 1)
y = np.linspace(*bins[1][[0, -1]], num=100 + 1)

XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])

values = {"III": {"llh": grbllh_ic86III, "cut": score_cut_86III},
          "II": {"llh": grbllh_ic86II, "cut": score_cut_86II}}

for axi, key in zip(ax, list(values.keys())):
    zz = values[key]["llh"]._soverb_energy(xx, yy)
    ZZ = zz.reshape(XX.shape)
    img = axi.pcolormesh(XX, YY, np.log10(ZZ), cmap="coolwarm",
                         vmin=-6, vmax=6)
    
    cbar = plt.colorbar(ax=axi, mappable=img)
    cbar.set_label("log10(ratio)")

    axi.set_xlabel("sin(dec)")
    axi.set_ylabel("muex in log10(GeV)")
    axi.set_title("Energy PDF for IC86{}. ".format(key) +
                 "score_cut: > {:.2f}".format(values[key]["cut"]))

fig.tight_layout()
# plt.savefig("./fig_mc_energy_pdf/score_cut_" +
#             "{:.2f}_mc_energy_pdf.png".format(score_cut), dpi=150)
plt.show()

## BG injectors

### 86III

In [ ]:
# Choose your rate function
rate_func_obj = RateFunc.Sinus1yrConstRateFunction(rndgen_86III)

# Make the injector based on the rate function
bg_rate_inj_86III = BGRateInj.RunlistBGRateInjector(
    rate_func_obj, goodruns_86III, random_state=rndgen_86III)

# Fit the injector to make it usable
times_86III = np.delete(data_pdf_86III["timeMJD"], [39023, 52880, 63524])
rate_func_86III = bg_rate_inj_86III.fit(T=times_86III, x0=None,
                                        remove_zero_runs=False)

# Now the evt injector for the other attributes
# bg_inj = BGInj.MRichmanBGInjector(random_state=rndgen)
bg_inj_86III = BGInj.DataBGInjector(random_state=rndgen_86III)

if type(bg_inj_86III) == BGInj.DataBGInjector:
    bg_inj_86III.fit(X=data_pdf_86III)
elif type(bg_inj_86III) == BGInj.MRichmanBGInjector:
    edges = np.vstack(([1., -0.1, 0.], [5., np.pi / 2., np.deg2rad(10.)])).T
    mr_nbins = 10
    all_bins = bg_inj_86III.fit(X=data_pdf, nbins=mr_nbins, minmax=False)

### 86II

In [ ]:
# Choose your rate function
rate_func_obj = RateFunc.Sinus1yrConstRateFunction(rndgen_86II)

# Make the injector based on the rate function
bg_rate_inj_86II = BGRateInj.RunlistBGRateInjector(
    rate_func_obj, goodruns_86II, random_state=rndgen_86II)

# Fit the injector to make it usable
out_times = [1455, 7030, 8863, 13724, 17655, 20423, 22191, 26127, 28365,
             45350, 48338, 53590, 104330, 120422, 123279]
times_86II = np.delete(data_pdf_86II["timeMJD"], out_times)
rate_func_86II = bg_rate_inj_86II.fit(T=times_86II, x0=None,
                                      remove_zero_runs=False)

# Now the evt injector for the other attributes
# bg_inj = BGInj.MRichmanBGInjector(random_state=rndgen)
bg_inj_86II = BGInj.DataBGInjector(random_state=rndgen_86II)

if type(bg_inj_86II) == BGInj.DataBGInjector:
    bg_inj_86II.fit(X=data_pdf_86II)
elif type(bg_inj_86II) == BGInj.MRichmanBGInjector:
    edges = np.vstack(([1., -0.1, 0.], [5., np.pi / 2., np.deg2rad(10.)])).T
    mr_nbins = 10
    all_bins = bg_inj_86II.fit(X=data_pdf, nbins=mr_nbins, minmax=False)

### Multi

In [ ]:
multibginj = BGInj.MultiBGInjector()
multibgrateinj = BGRateInj.MultiBGRateInjector()

In [ ]:
multibginj.add_injector("86II", bg_inj_86II)
multibginj.add_injector("86III", bg_inj_86III)

multibgrateinj.add_injector("86II", bg_rate_inj_86II)
multibgrateinj.add_injector("86III", bg_rate_inj_86III)

BGInjector

In [ ]:
nsamples = {"86II": 10, "86III": 5}
bgsam = multibginj.sample(nsamples)
bgsam

BGRateInjector

In [ ]:
t = {"86III": grb_srcs_IC86III["t"], "86II": grb_srcs_IC86II["t"]}
trange = {"86III": [[0, dt1] for dt1 in grb_srcs_IC86III["dt1"]],
          "86II": [[0, dt1] for dt1 in grb_srcs_IC86II["dt1"]]}
tsam = multibgrateinj.sample(t=t, trange=trange)
tsam

## Signal Injector tests

In [ ]:
siginj = SigInj.SignalInjector(gamma=2.)
mcs = {"86III": sim_pdf_86III, "86II": sim_pdf_86II}
srcs = {"86III": grb_srcs_IC86III, "86II": grb_srcs_IC86II}
siginj.fit(srcs=srcs, MC=mcs, exp_names=data_pdf_86III.dtype.names)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj.mu2flux(1)))

In [ ]:
gen = siginj.sample(mean_mu=10)
sam = next(gen)

In [ ]:
sam[1]["86III"].dtype.names

### Tests

Expect same result when plugging in the same sample and the same sources two times.
We have half the weights, but each event twice which cancels in effects.

In [ ]:
siginj_2 = SigInj.SignalInjector(gamma=2.)
mcs = {"86III": sim_pdf_86III, "86II": sim_pdf_86III}
srcs = {"86III": grb_srcs_IC86III, "86II": grb_srcs_IC86III}
siginj_2.fit(srcs=srcs, MC=mcs, exp_names=data_pdf_86III.dtype.names)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj_2.mu2flux(1)))

In [ ]:
siginj_1 = SigInj.SignalInjector(gamma=2.)
mcs = {"86III": sim_pdf_86III}
srcs = {"86III": grb_srcs_IC86III}
siginj_1.fit(srcs=srcs, MC=mcs, exp_names=data_pdf_86III.dtype.names)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj_1.mu2flux(1)))

In [ ]:
print("Both fluxes equal: ", np.isclose(siginj_1.mu2flux(1),
                                        siginj_2.mu2flux(1)))

Is single sample wrapper still working?

In [ ]:
siginj = SigInj.SignalInjector(gamma=2.)
siginj.fit(srcs=grb_srcs_IC86III, MC=sim_pdf_86III,
           exp_names=data_pdf_86III.dtype.names)
print("1 event flux: {:.6f} GeV^-1cm^-2".format(siginj.mu2flux(1)))

In [ ]:
gen = siginj.sample(mean_mu=10)
sam = next(gen)

In [ ]:
sam[1].dtype.names